## Autograd

In [76]:
import torch
from torch import nn

In [114]:
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()

        # self.weights = torch.rand(2, 2, requires_grad=True)
        self.linear1 = nn.Linear(2, 2)
        self.linear2 = nn.Linear(2, 2)
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        return torch.softmax(x, dim=1)

In [150]:
model = SimpleModel()

In [151]:
gt = torch.tensor([1, 0]).unsqueeze(0)
x = torch.rand(2).unsqueeze(0)
out = model(x)
gt, x, out

(tensor([[1, 0]]),
 tensor([[0.1570, 0.1996]]),
 tensor([[0.5020, 0.4980]], grad_fn=<SoftmaxBackward>))

In [152]:
model.linear1.weight, model.linear2.weight

(Parameter containing:
 tensor([[-0.1388, -0.6964],
         [-0.1724, -0.3032]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.4651,  0.5530],
         [-0.7043,  0.5696]], requires_grad=True))

In [153]:
r = torch.autograd.grad(out, model.linear2.weight, gt, only_inputs=False, create_graph=True)[0]
r

/home/pdpino/medical-ai/pyenv3.6/lib/python3.6/site-packages/torch/autograd/__init__.py:192: UserWarning: only_inputs argument is deprecated and is ignored now (defaults to True). To accumulate gradient for other parts of the graph, please use torch.autograd.backward.
  warnings.warn("only_inputs argument is deprecated and is ignored now "


tensor([[ 0.1351, -0.1982],
        [-0.1351,  0.1982]], grad_fn=<TBackward>)

In [149]:
model.linear2.weight.grad, out.grad

/home/pdpino/medical-ai/pyenv3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  """Entry point for launching an IPython kernel.


(None, None)

In [137]:
gt_map = torch.ones(r.size())
loss = (r - gt_map).sum()
loss.backward()

In [65]:
loss = (gt - out)
loss

tensor([0.5733, 0.5500], grad_fn=<SubBackward0>)

In [66]:
out.retain_grad()

In [69]:
model.weights.grad, x.grad, out.grad

(tensor([[-0.3334, -0.2032],
         [-0.3334, -0.2032]]),
 None,
 tensor([-1., -1.]))

In [68]:
loss.backward(torch.ones(out.size()), create_graph=True)

## Shared interest data files

In [1]:
import torch
import json

In [8]:
fpath = '~/software/shared-interest/data/examples/data_cxr14.json' # dogs

In [9]:
with open(fpath) as f:
    data = json.load(f)
len(data)

9

In [10]:
data.keys()

dict_keys(['fname', 'image', 'bbox', 'saliency', 'label', 'prediction', 'ground_truth_coverage', 'explanation_coverage', 'iou'])

In [13]:
set(type(a) for a in data['label'].values())

{str}

In [14]:
len(data['label'])

5045

## Debug MedicalLabelerCorrectness

In [6]:
import numpy as np
import torch
from torch.nn.utils.rnn import pad_sequence

In [7]:
%run ../metrics/report_generation/labeler_correctness/cache.py
%run ../metrics/report_generation/labeler_correctness/light_labeler.py
%run ../metrics/report_generation/labeler_correctness/__init__.py

In [8]:
%run ../datasets/vocab/__init__.py

In [9]:
%run ../datasets/iu_xray.py
%run ../utils/nlp.py

In [10]:
dataset = IUXRayDataset('all')
len(dataset)

7426

In [11]:
report_reader = ReportReader(dataset.get_vocab())

In [12]:
chexpert_labeler = ChexpertLightLabeler(dataset.get_vocab())
chexpert_labeler

In [15]:
reports = pad_sequence([
    torch.tensor(dataset[idx+2].report)
    for idx in [100, 200, 300, 400]
], batch_first=True)
reports

tensor([[ 32,  11,  50,   4,  52,  10,  13,   4,  17,  28,   4,   0,   0,   0,
           0],
        [ 40,  13,  52,  10,   4, 151,  43, 571,   4,  32,  11,  50,   4,   0,
           0],
        [ 52,  10,  16,  13,   8,  32,  11,  50,   4,  17,  27,  23, 563,   4,
           0],
        [ 52,  10,  13,   8,  32,  11,  50,   4, 300, 727,  63, 135,  68, 312,
           4]])

In [9]:
metric = MedicalLabelerCorrectness(chexpert_labeler)
metric

In [18]:
metric.reset()
metric.update((reports, reports))
metric.compute()

{'acc': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'prec': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]),
 'recall': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]),
 'spec': array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'npv': array([0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])}

In [11]:
[report_reader.idx_to_text(r) for r in reports]

['xxxx change . no active cardiopulmonary disease . thoracolumbar scoliosis . PAD PAD PAD PAD',
 'chest . no active disease . right shoulder . healing scapular fracture . PAD PAD',
 'the cardiac contours are normal . the lungs are clear . thoracic spondylosis . PAD',
 'heart size is normal and the lungs are clear . no nodules or masses .']